In [1]:
import extFunc
import numpy as np
import os
from importlib import import_module
import copy 
import pandas as pd
import sys
import cobra
import pickle


In [14]:
#Load Models if not already in RAM

#nameList = ['iJN1462_GLC_UR10_9.xml' , 'iJN1462_GLC_UR6_3.xml', 'iJN1462_GLN_UR5_1.xml', 'iJN1462_GLC_UR7_3.xml' , 'iJN1462_OCT_UR3_4.xml']
nameList = ['iJN1462_GLC_UR7_3.xml' , 'iJN1462_OCT_UR3_4.xml']


if 'modelDict' not in locals():
    modelDict = extFunc.ImportFunction(nameList)


In [15]:
class ModelComparison():
    '''
    This Class can be used to compare multiple modelversions 
    and to get an overview of the C-Flux routes and 
    the corresponding fva
    input  = list of models
    '''

    def __init__(self, modelDict: "modelDict", **kwargs):
        self.modelDict = modelDict
        self.solutionDict = {}
        self.summaryDict = {}
        self.dataFrameDict = {}
        self.generate()

    # collect the objects of interest
    def generate(self):
        for model in self.modelDict.keys():
            self.solutionDict[model] = self.modelDict[model].optimize()
            self.summaryDict[model] = self.modelDict[model].summary(
                solution=self.solutionDict[model], fva=.99)
            self.dataFrameDict[model] = {
                "uptake":   pd.read_html(self.summaryDict[model].to_html())[0].sort_values('C-Flux', ascending=False),
                "secretion":   pd.read_html(self.summaryDict[model].to_html())[1].sort_values('C-Flux', ascending=False)
            }

    def mySummary(self,nrRea=3):
        dfList, keyList = [], []
        
        for model in self.modelDict.keys(): 
            theMatrix   = [ self.dataFrameDict[model]['uptake'].iloc[0:nrRea].reset_index(drop=True),
                            self.dataFrameDict[model]['secretion'].iloc[0:nrRea].reset_index(drop=True) ] 
        
            modelFrame=pd.concat(
                [pd.concat(theMatrix,
                keys=['Uptake', 'Secretion'], axis=1)],
                keys=[model]
                ) 
            dfList.append(modelFrame)
            keyList.append(model)

        return pd.concat(dfList,axis=0)

In [16]:
if 'comp'  not in locals():
    comp = ModelComparison(modelDict=modelDict)

In [17]:
comp.mySummary(nrRea=5)

Uptake                           \
                      Metabolite      Reaction       Flux   
iJN1462_GLC_UR10_9 0   glc__D[e]   EX_glc__D_e  10.900000   
                   1      ca2[e]      EX_ca2_e   0.002422   
                   2      na1[e]      EX_na1_e   0.002019   
                   3      zn2[e]      EX_zn2_e   0.001614   
                   4      so4[e]      EX_so4_e   0.106600   
iJN1462_GLC_UR6_3  0   glc__D[e]   EX_glc__D_e   6.300000   
                   1  pqqA_kt[c]  SK_pqqA_kt_c   0.000137   
                   2      na1[e]      EX_na1_e   0.002168   
                   3      zn2[e]      EX_zn2_e   0.001734   
                   4      so4[e]      EX_so4_e   0.114500   
iJN1462_GLN_UR5_1  0     glcn[e]     EX_glcn_e   5.100000   
                   1      ca2[e]      EX_ca2_e   0.002005   
                   2     mobd[e]     EX_mobd_e   0.001548   
                   3      zn2[e]      EX_zn2_e   0.001337   
                   4      so4[e]      EX_so4_e   0.088260   
iJN1462_GLC_UR7_3  0   glc__D[e]   EX_glc__D_e   7.300000   
                   1  pqqA_kt[c]  SK_pqqA_kt_c   0.000159   
                   2      na1[e]      EX_na1_e   0.002515   
                   3      zn2[e]      EX_zn2_e   0.002011   
                   4      so4[e]      EX_so4_e   0.132800   
iJN1462_OCT_UR3_4  0     octa[e]     EX_octa_e   3.400000   
                   1      ca2[e]      EX_ca2_e   0.001231   
                   2       cl[e]       EX_cl_e   0.001231   
                   3      zn2[e]      EX_zn2_e   0.000820   
                   4      so4[e]      EX_so4_e   0.054170   

                                                                  Secretion  \
                                       Range C-Number   C-Flux   Metabolite   
iJN1462_GLC_UR10_9 0           [10.84; 10.9]        6  100.00%      glcn[e]   
                   1    [0.002398; 0.002422]        0    0.00%   2dhglcn[e]   
                   2    [0.001998; 0.002019]        0    0.00%       co2[e]   
                   3    [0.001598; 0.001614]        0    0.00%      icit[e]   
                   4         [0.1055; 0.269]        0    0.00%   mtsoxin[e]   
iJN1462_GLC_UR6_3  0            [6.237; 6.3]        6   99.99%       co2[e]   
                   1  [0.0001359; 0.0001369]       14    0.01%     5drib[c]   
                   2    [0.002147; 0.002168]        0    0.00%      amob[c]   
                   3     [0.001717; 0.00173]        0    0.00%    hom__L[e]   
                   4         [0.1134; 0.289]        0    0.00%      meoh[e]   
iJN1462_GLN_UR5_1  0            [5.049; 5.1]        6  100.00%       co2[e]   
                   1    [0.001985; 0.001997]        0    0.00%     5drib[c]   
                   2    [0.001533; 0.001542]        0    0.00%      amob[c]   
                   3    [0.001323; 0.001327]        0    0.00%      icit[e]   
                   4       [0.08737; 0.2227]        0    0.00%   mtsoxin[e]   
iJN1462_GLC_UR7_3  0            [7.227; 7.3]        6   99.99%       co2[e]   
                   1  [0.0001576; 0.0001588]       14    0.01%     5drib[c]   
                   2     [0.00249; 0.002515]        0    0.00%      amob[c]   
                   3    [0.001991; 0.002006]        0    0.00%    hom__L[e]   
                   4        [0.1315; 0.3352]        0    0.00%      meoh[e]   
iJN1462_OCT_UR3_4  0              [3.4; 3.4]        8  100.00%  R_3hocta[e]   
                   1    [0.001218; 0.001231]        0    0.00%        ac[e]   
                   2    [0.001218; 0.001231]        0    0.00%       co2[e]   
                   3  [0.0008122; 0.0008204]        0    0.00%     5drib[c]   
                   4           [0.05363; 10]        0    0.00%      amob[c]   

                                                                          \
                           Reaction       Flux                     Range   
iJN1462_GLC_UR10_9 0      EX_glcn_e  -2.800000            [-2.861; -2.8]   
                

In [18]:
#comp.summaryDict['iJN1462_OCT_UR3_4']._display_flux(summaryDict['iJN1462_OCT_UR3_4'])

















Trash Code

'tets'
#comp.solutionDict['iJN1462_GLC_UR10_9'].fluxes.get_by_id('EX_glc__D_e')
#comp.solutionDict['iJN1462_GLC_UR10_9'].fluxes.get_by_id('EX_octa_e')
#comp.solutionDict[['iJN1462_GLC_UR10_9']]
#b = comp.solutionDict['iJN1462_GLC_UR10_9'].fluxes['EX_glc__D_e']
#comp.solutionDict['iJN1462_GLC_UR10_9'].fluxes.get_by_id('EX_octa_e')
#a = comp.solutionDict['iJN1462_GLC_UR10_9']
#comp.modelDict['iJN1462_GLC_UR10_9'].summary()
'''
def createOutputWrapper(modelList):

    pass
    for model in modelList:
        tableForDisppay.join(modelList)      

    return tableForDisppay

def createOutput(model):
    sol = model.optimize()
    sum = model.summary(solution = sol,fva=.99)
    
    #biomass = solution.reac ('BIOMASS_KT2440_WT3') 
    print(biomass)
'''
    

#createOutput(modelList)       
#das gesamte Paket als 
#df = iJN1462_OCT_UR3_4.summary().to_frame()

#iJN1462_OCT_UR3_4.tolerance = 1e-1
#iJN1462_OCT_UR3_4.summary()
#dhtml =de.to_html()
#df = pd.read_html(dhtml)
#pd.DataFrame(df[1]).sort_values('C-Flux',ascending=False)
#df[1]
#
#de
#iJN1462_OCT_UR3_4.summary()
#iJN1462_OCT_UR3_4._tolerance = 2
#solutionSummaryObject = iJN1462_OCT_UR3_4.summary(fva=.99)


#pd.DataFrame(pd.read_html(dhtml)[1]).sort_values('C-Flux',ascending=False)


#iJN1462_OCT_UR3_4.reactions.get_by_id('biomass')
#createOutput(iJN1462_GLC_UR10_9)
